In [ ]:
import pandas as pd
import numpy as np
import pyspark
#from pyspark.logger import PySparkLogger
from pyspark.sql import SparkSession
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
import matplotlib.pyplot as plt 


In [ ]:
spark= SparkSession.builder.appName('Test').config("spark.sql.debug.maxToStringFields", 100).getOrCreate()
df_spark= spark.read.option('header','true').option('inferSchema','true').csv('/Users/gyanapradhan/Desktop/NY_censusdata.csv')

In [ ]:
df_spark.show(2,truncate=False)

In [ ]:
df=pd.read_csv("/Users/gyanapradhan/Desktop/NY_censusdata.csv")
df.head(10)


In [ ]:
df.columns


In [ ]:
df_subset= df[['tree_id', 'block_id', 'created_at', 'tree_dbh', 'stump_diam',
       'curb_loc', 'status', 'health', 'sidewalk', 'user_type', 'problems', 'root_stone',
       'root_grate', 'trunk_wire', 'trnk_light', 'trnk_other',
       'brch_light', 'brch_shoe', 'brch_other']]

In [ ]:
df_subset


In [ ]:
df_subset.isnull().sum()

In [ ]:
df_subset[df_subset['health'] .isnull()]

In [ ]:
df_subset.describe()


In [ ]:
df_subset.dtypes


In [ ]:
df_subset.hist(bins=50)

In [ ]:
df_subset[df_subset['tree_dbh']>=30]

In [ ]:
bigtrees=df_subset[df_subset['tree_dbh']>= 50]

In [ ]:
bigtrees

In [ ]:
bigtrees[['tree_id','tree_dbh']].plot(kind='scatter',x='tree_id',y='tree_dbh')

In [ ]:
Problem_trees= df_subset[['root_stone', 'root_grate', 'trunk_wire', 'trnk_light', 'trnk_other',
       'brch_light', 'brch_shoe', 'brch_other']]
Problem_trees.apply(pd.Series.value_counts)


In [ ]:

df_subset['health'].fillna('None',inplace=True)
df_subset['problems'].fillna('Good', inplace=True)
df_subset['sidewalk'].fillna('NoDamage',inplace=True)

In [ ]:
df_subset.isna().sum()

In [ ]:
df_subset_aliveTree = df_subset[df_subset['status']=='Alive']
df_subset_tree_dead_or_stump = df_subset[(df_subset['status']== 'Dead') | (df_subset['status']=='stump')]

In [ ]:
df_stats=df_subset_aliveTree.groupby('created_at')['tree_dbh'].describe().reset_index()[['created_at','25%','75%']]


In [ ]:
df_subset_alivetree_new = df_subset_aliveTree.merge(df_stats,on='created_at',how= 'left')

In [ ]:
mask= df_subset_alivetree_new ['tree_dbh']< df_subset_alivetree_new ['25%']
df_subset_alivetree_new.loc[mask,['tree_dbh']] = df_subset_alivetree_new ['25%']

In [ ]:
df_subset_alivetree_new

In [ ]:
mask= df_subset_alivetree_new ['tree_dbh']> df_subset_alivetree_new ['75%']
df_subset_alivetree_new.loc[mask,['tree_dbh']] = df_subset_alivetree_new ['75%']

In [ ]:
df_subset_alivetree_new[df_subset_alivetree_new.problems == 'Good']

In [ ]:
health_good= (df_subset_alivetree_new.problems == 'Good')
df_subset_alivetree_new[health_good]